In [7]:
from census import Census
import pandas

API_KEY = "f88810dbb816ce76eb26fc06cb5d34c2ea1f96ad"

# Example census API call

In [8]:
c = Census(API_KEY)
data = pandas.DataFrame(c.acs5.get(('C24050_035E'), {'for': 'county:*'}))
data

,C24050_035E,state,county
0,9.0,17,051
1,23.0,17,107
2,35.0,17,165
3,400.0,17,097
4,18.0,17,127
...,...,...,...
3215,21.0,47,033
3216,0.0,47,095
3217,211.0,47,093
3218,184.0,53,005


# Format employment data

In [6]:
def get_percent(df, column):
    df[column] = df[column] / df["AllIndustries"] * 100
def geoid_to_county(geoid):
    return geoid.split("0500000US")[1][2:5]
def geoid_to_state(geoid):
    return geoid.split("0500000US")[1][0:2]

demographics = pandas.read_csv("demographics.csv", dtype={"STATEFP":str,"COUNTYFP":str})
data = pandas.read_csv(
    "ACSST5Y2019.S2403_2021-04-02T133618/ACSST5Y2019.S2403_data_with_overlays_2021-04-01T195355.csv",
    usecols=[
    "id",
    "Estimate!!Total!!Civilian employed population 16 years and over",
    "Estimate!!Total!!Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining:!!Agriculture, forestry, fishing and hunting",
    "Estimate!!Total!!Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining:!!Mining, quarrying, and oil and gas extraction",
    "Estimate!!Total!!Civilian employed population 16 years and over!!Construction",
    "Estimate!!Total!!Civilian employed population 16 years and over!!Educational services, and health care and social assistance:!!Health care and social assistance",
])
renamed_cols = {
    "id": "geoid",
    "Estimate!!Total!!Civilian employed population 16 years and over":"AllIndustries",
    "Estimate!!Total!!Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining:!!Agriculture, forestry, fishing and hunting":"AgricultureForestryFishingAndHunting",
    "Estimate!!Total!!Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining:!!Mining, quarrying, and oil and gas extraction":"MiningQuarryingAndOilAndGasExtraction",
    "Estimate!!Total!!Civilian employed population 16 years and over!!Construction":"Construction",
    "Estimate!!Total!!Civilian employed population 16 years and over!!Educational services, and health care and social assistance:!!Health care and social assistance":"HealthCareAndSocialAssistance"
}
data = data.rename(columns=renamed_cols)
cols = ["AgricultureForestryFishingAndHunting","MiningQuarryingAndOilAndGasExtraction","Construction","HealthCareAndSocialAssistance"]
for col in cols:
    get_percent(data, col)
data["STATEFP"] = data["geoid"].apply(geoid_to_state)
data["COUNTYFP"] = data["geoid"].apply(geoid_to_county)
del data["geoid"]
data.round(2).to_csv("census_employment.csv", index=False)
data["geoid"] = data["STATEFP"]+data["COUNTYFP"]
data["geoid"] = data["geoid"].astype(str)
demographics["geoid"] = data["STATEFP"]+data["COUNTYFP"]
demographics["geoid"] = demographics["geoid"].astype(str)
data

,AllIndustries,AgricultureForestryFishingAndHunting,MiningQuarryingAndOilAndGasExtraction,Construction,HealthCareAndSocialAssistance,STATEFP,COUNTYFP,geoid
0,24522,0.517902,0.354783,6.072099,11.267433,01,001,01001
1,95091,0.952772,0.257648,8.585460,12.600562,01,003,01003
2,8413,5.717342,0.000000,6.810888,9.437775,01,005,01005
3,8387,1.967330,1.895791,9.848575,13.687850,01,007,01007
4,21917,1.446366,0.634211,9.718483,14.395218,01,009,01009
...,...,...,...,...,...,...,...,...
3215,14915,0.918538,0.120684,5.786121,14.046262,72,145,72145
3216,2467,0.567491,0.000000,15.849210,8.147548,72,147,72147
3217,6579,3.070375,0.000000,6.292750,12.144703,72,149,72149
3218,8611,0.673557,0.000000,4.308443,10.997561,72,151,72151


In [3]:
demographics

,STATEFP,COUNTYFP,discuss,reducetax,CO2limits,localofficials,governor,congress,president,corporations,...,MiningEPercentage,ConstructionEPercentage,HealthcareandsocialassistanceEPercentage,PercentPopulationUnder18,PercentPopulationOver65,PercentNonwhite,PercentofPopulationBelowPovertyLevel,UnemploymentRate,Populationpersquaremile2010,geoid
0,01,001,26.86,53.31,56.68,49.20,48.81,50.27,52.92,60.27,...,0.28,9.44,7.04,24.55,14.25,23.40,13.60,2.7,91.8,01001
1,01,003,30.09,52.47,54.75,49.34,50.45,52.72,53.13,61.79,...,0.27,11.14,7.92,22.13,19.02,13.47,11.67,2.7,114.6,01003
2,01,005,28.12,63.44,64.92,58.36,57.78,60.07,58.82,66.03,...,0.95,0.00,5.56,21.04,17.45,53.44,24.17,3.8,31.0,01005
3,01,007,25.78,53.88,55.50,49.26,48.53,49.88,52.09,59.41,...,0.00,15.97,0.00,20.94,15.07,23.28,13.88,3.1,36.8,01007
4,01,009,26.00,48.24,51.99,44.37,44.17,46.28,50.64,56.87,...,0.00,14.47,6.14,23.52,17.35,4.55,15.47,2.7,88.9,01009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56,037,28.29,47.79,41.48,47.75,45.74,48.60,54.07,66.20,...,20.02,8.91,4.30,26.85,9.74,7.14,11.84,3.9,4.2,56037
3138,56,039,47.80,69.54,72.57,58.26,57.20,66.11,60.09,75.35,...,0.00,11.55,3.56,19.35,12.70,9.82,6.81,2.7,5.3,56039
3139,56,041,34.17,53.41,56.55,45.80,45.07,50.17,51.23,62.37,...,8.38,12.11,10.20,29.60,11.69,5.98,14.68,3.9,10.1,56041
3140,56,043,34.33,51.17,55.50,45.38,44.49,48.43,51.24,62.80,...,5.42,7.43,10.59,24.60,20.03,10.01,12.59,3.9,3.8,56043


In [4]:
end_result = pandas.merge(demographics, data, on="geoid", how="outer")
end_result["COUNTYFP"] = end_result["COUNTYFP_y"]
end_result["STATEFP"] = end_result["STATEFP_y"]
del end_result["COUNTYFP_y"]
del end_result["STATEFP_y"]
del end_result["COUNTYFP_x"]
del end_result["STATEFP_x"]
del end_result["geoid"]
end_result.round(2).to_csv("test.csv")

In [10]:
c.acs5.get('C24050_035E', {'for': 'county:*'})

[{'C24050_035E': 9.0, 'state': '17', 'county': '051'},
 {'C24050_035E': 23.0, 'state': '17', 'county': '107'},
 {'C24050_035E': 35.0, 'state': '17', 'county': '165'},
 {'C24050_035E': 400.0, 'state': '17', 'county': '097'},
 {'C24050_035E': 18.0, 'state': '17', 'county': '127'},
 {'C24050_035E': 6.0, 'state': '17', 'county': '017'},
 {'C24050_035E': 7.0, 'state': '18', 'county': '069'},
 {'C24050_035E': 0.0, 'state': '18', 'county': '181'},
 {'C24050_035E': 3.0, 'state': '18', 'county': '075'},
 {'C24050_035E': 12.0, 'state': '18', 'county': '145'},
 {'C24050_035E': 0.0, 'state': '18', 'county': '153'},
 {'C24050_035E': 43.0, 'state': '18', 'county': '157'},
 {'C24050_035E': 12.0, 'state': '18', 'county': '057'},
 {'C24050_035E': 19.0, 'state': '18', 'county': '005'},
 {'C24050_035E': 0.0, 'state': '18', 'county': '049'},
 {'C24050_035E': 6.0, 'state': '18', 'county': '113'},
 {'C24050_035E': 105.0, 'state': '18', 'county': '019'},
 {'C24050_035E': 41.0, 'state': '18', 'county': '063'}